## Convert .bin drawing to .npy drawing

### Import

In [64]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from random import randint
import re

### Load data

In [65]:
object_file = open("object.txt", "r")
objects = object_file.readlines()
object_file.close()
N_CLASSES = len(objects)
CLASSES = {}
for idx, obj in enumerate(objects):
    CLASSES[idx] = obj.replace('\n', '')
# print(CLASSES)



In [66]:
def extract_label(file_path):
    """
    Extract the variable from a string in the format npy_data\{variable}.npy.

    Parameters:
    - file_path: String containing the path.

    Returns:
    - str: Extracted variable.
    """
    match = re.search(r'npy_data\\(.*?)\.npy', file_path)
    if match:
        variable = match.group(1)
        return variable
    else:
        return None
    
def get_key_by_value(dictionary, target_value):
    """
    Get the key associated with a specific value in a dictionary.

    Parameters:
    - dictionary: The input dictionary.
    - target_value: The value to search for.

    Returns:
    - key: The key associated with the target value, or None if not found.
    """
    for key, value in dictionary.items():
        if value == target_value:
            return key
    return None

In [67]:
nparrs = np.load("./npy_data/apple.npy")

In [68]:
nparrs[0].shape

(784,)

In [69]:
# Get random 1000 drawings from each object
def load_drawings(root, reshaped=False):
    all_files = glob.glob(os.path.join(root, '*.npy'))
    
    data = []
    label_data = []
    for index, file in enumerate(all_files):
        extracted_label = extract_label(file)
        np_arrays = np.load(file)
        label_arrays = []
        if reshaped:
            new_arrays = []
            new_label_arrays = []
            for idx in range(len(np_arrays)):
                rot_coeff = randint(0,3)
                label_index = get_key_by_value(CLASSES, extracted_label)
                if label_index == None:
                    continue
                # Reshape into a picture 28x28
                np_arr = np.reshape(np_arrays[idx] ,(28,28))
                # Rotate for better model
                np_arr = np.rot90(np_arr, rot_coeff)
                # Add another dimension for CNN network
                np_arr = np.reshape(np_arr, (28,28,1))
                new_arrays.append(np_arr)
                new_label_arrays.append(label_index)
            np_arrays = new_arrays
            label_arrays = new_label_arrays
        data.append(np_arrays)
        label_data.append(label_arrays)
    return data, label_data

In [70]:
data, label_data = load_drawings('npy_data', reshaped=True)

In [71]:
# print(len(label_data))

In [72]:
# print(len(data))

In [73]:
def visualize(nparr):
    img = Image.fromarray(nparr.reshape(28,28))
    img.show(title="visualize array")

In [74]:
visualize(data[0][100])

In [75]:
def set_lim(np_arrays, label_data_arrays, lim):
    lim_arr = []
    lim_labels = []
    for arr_index in range(len(np_arrays)):
        i = 0
        data_array = np_arrays[arr_index]
        label_array = label_data_arrays[arr_index]
        for index in range(len(data_array)):
            if i == lim:
                break
            lim_arr.append(data_array[index])
            lim_labels.append(label_array[index])
            i += 1
    return lim_arr, lim_labels

In [76]:
data, labels = set_lim(data, label_data, 1000)

In [77]:
# print(len(data))
# print(len(labels))
# print(labels[2000:2010])

In [78]:
def normalize(nparr):
    return np.interp(nparr, [0, 255], [-1,1])

In [79]:
def denormalize(nparr):
    return np.interp(nparr, [-1, 1], [0, 255])

In [80]:
data = normalize(data)

# print(len(labels))

In [81]:
data

array([[[[-1.        ],
         [-1.        ],
         [-1.        ],
         ...,
         [-1.        ],
         [-1.        ],
         [-1.        ]],

        [[-1.        ],
         [-1.        ],
         [-1.        ],
         ...,
         [-1.        ],
         [-1.        ],
         [-1.        ]],

        [[-1.        ],
         [-1.        ],
         [-1.        ],
         ...,
         [-1.        ],
         [-1.        ],
         [-1.        ]],

        ...,

        [[-1.        ],
         [-1.        ],
         [-1.        ],
         ...,
         [ 1.        ],
         [-0.27843137],
         [-1.        ]],

        [[-1.        ],
         [-1.        ],
         [-1.        ],
         ...,
         [ 0.74117647],
         [-0.84313725],
         [-1.        ]],

        [[-1.        ],
         [-1.        ],
         [-1.        ],
         ...,
         [-1.        ],
         [-1.        ],
         [-1.        ]]],


       [[[-1.        ],


## Training

In [82]:
from sklearn.model_selection import train_test_split as tts
from keras.utils import np_utils

#### Hot encoding

In [83]:
x_train, x_test, y_train, y_test = tts(data, labels, test_size=0.2)

In [84]:
Y_train = np_utils.to_categorical(y_train, N_CLASSES)
Y_test = np_utils.to_categorical(y_test, N_CLASSES)

In [85]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten


def conv(classes, input_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes, activation='softmax'))
    return model


In [86]:
model = conv(classes=N_CLASSES, input_shape=(28,28,1))

In [87]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [88]:
EPOCHS = 20
model.fit(np.array(x_train), np.array(Y_train), batch_size=32, epochs=EPOCHS, verbose=1)

Epoch 1/20


100/100 [==============================] - 2s 12ms/step - loss: 0.8790 - accuracy: 0.6391
Epoch 2/20
100/100 [==============================] - 1s 10ms/step - loss: 0.4941 - accuracy: 0.8378
Epoch 3/20
100/100 [==============================] - 1s 7ms/step - loss: 0.3731 - accuracy: 0.8813
Epoch 4/20
100/100 [==============================] - 1s 9ms/step - loss: 0.3073 - accuracy: 0.9041
Epoch 5/20
100/100 [==============================] - 1s 7ms/step - loss: 0.2669 - accuracy: 0.9116
Epoch 6/20
100/100 [==============================] - 1s 8ms/step - loss: 0.2371 - accuracy: 0.9187
Epoch 7/20
100/100 [==============================] - 1s 7ms/step - loss: 0.2044 - accuracy: 0.9316
Epoch 8/20
100/100 [==============================] - 1s 7ms/step - loss: 0.1901 - accuracy: 0.9356
Epoch 9/20
100/100 [==============================] - 1s 7ms/step - loss: 0.1645 - accuracy: 0.9419
Epoch 10/20
100/100 [==============================] - 1s 7ms/step - loss: 0.1517 - accuracy: 0.9494
Epoch 11

In [89]:
pred = model.predict(np.array(x_test))

25/25 [==============================] - 0s 3ms/step


In [90]:
score = 0

for i in range(len(pred)):
    if np.argmax(pred[i]) == y_test[i]:
        score+=1

acc = ((score+0.0)/len(pred)*100)
print("Accuracy: {acc}".format(acc = ((score+0.0)/len(pred)*100)))

Accuracy: 89.625


In [91]:
from datetime import datetime
# Get the current date and time
current_time = datetime.now()

# Format the date and time as a string (e.g., "2022-01-01_12-30-45")
formatted_time = current_time.strftime("%Y-%m-%d_%H-%M-%S")

# Combine the formatted time with a base file name
base_file_name = "./model/doodle"

file_name_with_timestamp = f"{base_file_name}.h5"

model.save(file_name_with_timestamp)

In [92]:
from keras.models import load_model

model = load_model("./model/doodle.h5")


In [93]:
def load_numpy_array(file_path):
    """
    Load a NumPy array from a file.

    Parameters:
    - file_path: File path to load the array from.

    Returns:
    - numpy array: Loaded NumPy array.
    """
    loaded_array = np.load(file_path)
    return loaded_array

# # Example Usage:
# file_path = "./user-data.npy"
# loaded_array = load_numpy_array(file_path)
# print("Loaded NumPy array:")
# visualize(denormalize(loaded_array))
# print(model.predict(loaded_array))
# pred = CLASSES[np.argmax(model.predict(loaded_array))]
# print("Predicted:", pred)

In [94]:
from random import randint
def visualize_and_predict():
    "selects a random test case and shows the object, the prediction and the expected result"
    n = randint(0, len(x_test))
    visualize(denormalize(np.reshape(x_test[n], (28, 28))))
    pred = CLASSES[np.argmax(model.predict(np.array([x_test[n]])))]
    actual = CLASSES[y_test[n]]
    print("Actual:", actual)
    print("Predicted:", pred)


### Evaluate model

In [95]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score)